### Importing Python Packages and Utility Functions

In [3]:
import pandas as pd
import numpy as np

import datetime
import os, sys
import importlib

import utils
importlib.reload(utils)
import nbformat

from utils import plot_series, plot_series_with_names, plot_series_bar
from utils import plot_dataframe
from utils import get_universe_adjusted_series, scale_weights_to_one, scale_to_book_long_short
from utils import generate_portfolio, backtest_portfolio
from utils import match_implementations

import plotly.graph_objects as go

### Data Loading and Structure

The dataset consists of three key files:


- **features.parquet**: A DataFrame containing **22 stock features** for each trading day from **2005 to 2025**, stored in a **columnar fashion**. For example, `features["macd"]` will return the `macd` of shape **(5068, 2167)**. All **22** features are guaranteed to have the same shape.

- **universe.parquet**: A DataFrame of the same shape as `features["macd"]`, containing `0` and `1` values, where `1` indicates that the stock is tradable on that day.

- **returns.parquet**: A DataFrame of shape **(3775, 2167)** containing **daily stock returns** from **2005 to 2019**. You will **never receive** the returns for the testing period i.e. from **2020 to 2025**.

### Data Organization:
- **Columns** represent stock identifiers, ranging from **1 to 2167** in increasing order.
  
- **Rows** represent trading days when the market was open.  


In [6]:
data_dir = "../qrt-quant-quest-iit-bombay-2025/"

features = pd.read_parquet(os.path.join(data_dir, "features.parquet"))
universe = pd.read_parquet(os.path.join(data_dir, "universe.parquet"))
returns = pd.read_parquet(os.path.join(data_dir, "returns.parquet"))


### Benchmark Strategy: Vectorized Portfolio Generation

In this step we will write a vectorized code to generate our strategy portfolio for all trading days at once without a `for` loop. For doing this we will use direct operations on feature dataframes. But, here's one caution. It is very easy to look into the future feature data when you're using dataframes to construct your portfolio at once.

Below is a vectorized implementation of the benchmark strategy [which you see on Kaggle]. Make sure to note the `alpha.shift(1)` operation in the code! This is to make sure you use only feature values upto the last trading day to construct today's portfolio weights.

You can change the code below to implement your own strategy. Since this vectorized implementation is faster, you can try various versions of your strategy and see their performances.

#### Inputs:
- `entire_features :pd.DataFrame`: Historical feature data (MultiIndex columns: Feature Names, Stock Identifiers).
- `universe: pd.DataFrame`: Binary DataFrame indicating tradable stocks per day.
- `start_date`, `end_date` :`str`: Backtest period in `'YYYY-MM-DD'` format.

#### Output:
- `portfolio(pd.DataFrame)`: Normalized portfolio weights for each stock per day in the specified date range.


In [ ]:
# A Benchmark Strategy for your reference: 
# This is the code used to generate the Benchmark submission you see in the Kaggle Leaderboard

# This strategy shows how you can combine different features 
def generate_portfolio_vectorized(
    entire_features: pd.DataFrame,
    universe: pd.DataFrame,
    start_date: str,
    end_date: str,
):
    # Validate date format
    try:
        start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
        cutoff_date = datetime.datetime.strptime('2005-01-01', '%Y-%m-%d')
    except ValueError:
        raise ValueError("start_date and end_date must be strings in 'YYYY-MM-DD' format.")

    # Ensure start_date is before end_date
    if start_dt >= end_dt:
        raise ValueError("start_date must be earlier than end_date.")

    # Ensure start_date is not before '2005-01-01'
    if start_dt < cutoff_date:
        raise ValueError("start_date must be later than '2005-01-01'.")

    # Get trading days within the date range
    trading_days = universe.index[(universe.index >= start_dt) & (universe.index <= end_dt)]

    if len(trading_days) == 0:
        raise ValueError("No Trading Days in the specified dates")
    
    portfolio = 0

    universe_boolean = universe.loc[:end_date].astype(bool)
    
    features_ = entire_features.loc[:end_date]

    signal1 = features["on_balance_volume"].shift(1)
    signal1 = signal1.where(universe_boolean, np.nan)
    signal1 = signal1.rank(axis=1, method="min", ascending=True)
    signal1 = signal1.sub(signal1.mean(axis=1), axis=0)
    signal1 = signal1.div(signal1.abs().sum(axis=1), axis=0)
    signal1 = signal1.fillna(0)

    signal2 = features["ultimate_oscillator"].shift(1)
    signal2 = signal2.where(universe_boolean, np.nan)
    signal2 = signal2.rank(axis=1, method="min", ascending=True)
    signal2 = signal2.sub(signal2.mean(axis=1), axis=0)
    signal2 = signal2.div(signal2.abs().sum(axis=1), axis=0)
    signal2 = signal2.fillna(0)

    portfolio = signal1 - signal2

    portfolio = portfolio.div(portfolio.abs().sum(axis=1), axis=0)
    
    return portfolio.fillna(0).loc[start_date:end_date]

### Generate your portfolio using the `generate_portfolio_vectorized` function you wrote above

In [ ]:
benchmark_portfolio_vectorized = generate_portfolio_vectorized(
    features,
    universe,
    "2005-01-03",
    "2025-02-07"
)

### Backtest your portfolio generated using vectorized code

Note that you can backtest your portfolio till `2019-12-31` since this is the last date in the training period. You don't have access to returns after this date. 

In [ ]:
sr_vectorized, pnl_vectorized = backtest_portfolio(benchmark_portfolio_vectorized.loc[:"2019"], returns.loc[:"2019"], universe.loc[:"2019"], True, True)

Gross Sharpe Ratio:  1.32
Net Sharpe Ratio:  1.11
Turnover %:  31.872


: 

### Benchmark Strategy: Iterative Portfolio Generation

Although the Vectorized Function generated the portfolio very quickly, it is very easy to look into the future data if you are not careful. For instance, remove the `shift(1)` operation and see the performance of the portfolio 😊. Hence, if your vectorized code has a forward bias [lookahead bias], you may see very high [or very low] sharpe ratios which may never be realised in real trading.

To avoid making these mistakes, we simulate our portfolio in a daily iterative fashion, where we call the `get_weights(features: pd.DataFrame, today_universe: pd.Series) -> dict[str, float]` function with **only** the past features data and the current day's trading universe. 

### Function Inputs:
- `features(pd.DataFrame)`:  
  - Contains various stock features indexed by date and stock identifiers.  
  - The features are structured in a **MultiIndex format**, where level 0 represents **feature names** (e.g., "macd", "volatility_60"), and level 1 represents **stock identifiers** (e.g., "1", "2", ..., "2167").  

- `today_universe(pd.Series)`:  
  - A series indicating which stocks can be traded on the current day.  
  - Contains **binary values (0 or 1)**, where **1** means a stock is **tradable**, and **0** means it is not.

You have to change this code, and write your own strategy code inside this function. Make sure it follows the same semantics as explained above.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Train a Linear Regression model (with PCA for dimensionality reduction)
def train_linear_regression(features: pd.DataFrame, returns: pd.DataFrame, num_components: int = 6) -> tuple[LinearRegression, PCA, StandardScaler]:
    """
    Train a Linear Regression model using historical features and returns with PCA.

    Parameters:
    -----------
    features : pd.DataFrame
        Historical feature data (MultiIndex columns: Feature Names, Stock Identifiers).
    returns : pd.DataFrame
        Historical returns data (columns: Stock Identifiers, rows: Dates).
    num_components : int
        Number of PCA components to use for dimensionality reduction.

    Returns:
    --------
    tuple[LinearRegression, PCA, StandardScaler]
        A trained Linear Regression model, PCA object, and StandardScaler object.
    """
    # Prepare the training data
    X = features.iloc[:-1].stack().reset_index()  # Use all but the last row of features
    X.columns = ["date", "feature", "stock", "value"]
    X = X.pivot(index=["date", "stock"], columns="feature", values="value").reset_index()

    y = returns.stack().reset_index()  # Flatten the returns DataFrame
    y.columns = ["date", "stock", "return"]

    # Merge features and target
    data = pd.merge(X, y, on=["date", "stock"]).dropna()

    # Split into features (X) and target (y)
    X_train = data.drop(columns=["date", "stock", "return"])
    y_train = data["return"]

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=min(num_components, X_train_scaled.shape[1]))
    X_train_pca = pca.fit_transform(X_train_scaled)

    # Train the Linear Regression model
    model = LinearRegression()
    model.fit(X_train_pca, y_train)

    return model, pca, scaler


# Linear Regression-based get_weights function
def get_weights_wrapper(features: pd.DataFrame, today_universe: pd.Series, model: LinearRegression, pca: PCA, scaler: StandardScaler) -> dict[str, float]:
    """
    Calculate stock weights for the portfolio on the current trading day using a Linear Regression model.

    Parameters:
    -----------
    features : pd.DataFrame
        A pandas DataFrame containing feature data.
        - Index: Datetime (chronological order).
        - Columns: MultiIndex with two levels:
          - Level 0: Feature names (e.g., "macd", "ichimoku", ..., "volatility_60").
          - Level 1: Stock identifiers (e.g., "1", "2", ..., "2167").

    today_universe : pd.Series
        A pandas Series indicating the stock universe for the current day.
        - Index: Stock identifiers.
        - Values: 0 or 1, where 1 means the stock is in the universe and 
          0 means it is not in the universe.

    model : LinearRegression
        A trained Linear Regression model.
    pca : PCA
        PCA object used for dimensionality reduction.
    scaler : StandardScaler
        StandardScaler object used for feature scaling.

    Returns:
    --------
    dict[str, float]
        A dictionary where:
        - Keys: Stock identifiers (strings).
        - Values: Weights/positions of the stocks (floats) for the current trading day.
    """
    if features.shape[0] == 0:
        return (today_universe * 1).replace(0, np.nan).dropna().fillna(0).to_dict()

    # Extract the last day's features
    last_day_features = features.iloc[-1].unstack().sort_index(axis=1)

    # Prepare the input data for prediction
    X_today = last_day_features.T.reset_index()
    X_today.columns = ["stock"] + list(last_day_features.index)
    X_today = X_today.set_index("stock")

    # Align with the universe
    X_today = X_today.loc[today_universe[today_universe == 1].index]

    # Standardize and apply PCA to today's features
    X_today_scaled = scaler.transform(X_today)
    X_today_pca = pca.transform(X_today_scaled)

    # Predict returns using the Linear Regression model
    predictions = model.predict(X_today_pca)

    # Create a signal DataFrame
    signal = pd.Series(predictions, index=X_today.index)
    signal = get_universe_adjusted_series(signal, today_universe)

    # Normalize the signal to satisfy constraints
    alpha = scale_to_book_long_short(signal)  # Dollar neutral and unit capital constraint
    alpha = alpha.clip(-0.1, 0.1)  # Maximum weight constraint
    alpha = alpha.replace(0, np.nan).dropna()  # Remove non-tradable stocks

    return alpha.to_dict()


# Train the Linear Regression model
model, pca, scaler = train_linear_regression(features, returns)

# Wrapper function for get_weights
def get_weights(features: pd.DataFrame, today_universe: pd.Series) -> dict[str, float]:
    return get_weights_wrapper(features, today_universe, model, pca, scaler)

/tmp/ipykernel_10534/2993271776.py:27: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Generate your portfolio using the `generate_portfolio` and `get_weights` function you wrote above

Since this function is iteratively called on every trading day, it takes a lot of time [about 40 mintues] to generate the entire portfolio dataframe from `2005-01-03` to `2025-02-07`. Hence, to show an example we call it only for a one year period from `2010-01-01` to `2010-12-31`.

In [ ]:
benchmark_portfolio = generate_portfolio(
    get_weights,
    features,
    universe,
    "2010-01-01",
    "2010-12-31",
)

  0%|          | 0/252 [00:00<?, ?it/s]


TypeError: get_weights() missing 1 required positional argument: 'model'

### Backtest your portfolio

Note that you can backtest your portfolio till `2019-12-31` since this is the last date in the training period. You don't have access to returns after this date. 

In [ ]:
sr, pnl = backtest_portfolio(benchmark_portfolio.loc["2010"], returns.loc["2010"], universe.loc["2010"], True, True)

Gross Sharpe Ratio:  -1.788
Net Sharpe Ratio:  -2.257
Turnover %:  52.181


You can also check the performance of your vectorized portfolio in this period to see if they match!

In [ ]:
sr, pnl = backtest_portfolio(benchmark_portfolio_vectorized.loc["2010"], returns.loc["2010"], universe.loc["2010"], True, True)

Gross Sharpe Ratio:  1.177
Net Sharpe Ratio:  0.866
Turnover %:  31.573


### Comparing Iterative and Vectorized Portfolio Implementations
This function evaluates **iterative** and **vectorized** portfolio generation methods by comparing their PnL correlation. If **correlation ≥ 0.98**, both implementations are considered equivalent.
#### Steps:
1. Selects a random start date.
2. Generates portfolios using both methods.
3. Backtests portfolios and computes PnLs.
4. Validates PnL correlation.
#### Criteria:
- **Pass:** Correlation **≥ 0.98** (implementations match).
- **Fail:** Correlation **< 0.98** (error raised).
#### Inputs:
- `contestant_get_weights`: Function for portfolio weights.
- `contestant_vectorized_portfolio`: A Pandas DataFrame containing portfolio weights generated using Vectorized Implementation
- `entire_features`: Feature data (MultiIndex columns).
- `universe`: Tradable stocks (binary).
- `returns`: Daily stock returns.
#### Output:
- Prints PnL correlation or raises an error if mismatched.

In [ ]:
match_implementations(get_weights, benchmark_portfolio_vectorized, features, universe, returns)

Starting to generate Iterative Portfolio


  0%|          | 0/41 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:06<00:00,  6.67it/s]
/home/ironman/Desktop/qrt-quant-quest-iit-bombay-2025/utils.py:301: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Iterative Portfolio Generated


ValueError: Correlation -0.6528310588550608 between PnL of Iterative and Vectorized Implementations is less than 0.98. Both Implementations don't match!

### Final Notes
- We recommend using vectorized code to test out your strategies. This will be easier and faster to run but make sure to `shift(1)` feature dataframes in order to avoid lookahead or forward bias.
- At the end when you have decided your final strategy that you want to submit for the competition, we advise you to write code for `get_weights` which will help iteratively generate your portfolio. 
- Finally, before submitting make sure to run `match_implementations` to make sure that both versions of your code produce the same portfolio
- If these two portfolios match, you can submit the one which was produced by `generate_portfolio_vectorized` without waiting for the iterative portfolio. You don't need to run the `generate_portfolio` function for 20 years!
- We will check that the submission you made on Kaggle matches with the portfolio generated by your code. If these two don't match, you will be eliminated from the competition. 

In [ ]:
# Submit this csv file on kaggle
benchmark_portfolio_vectorized.to_csv("submission.csv")

In [ ]:
print(universe.shape)

(5058, 2167)


In [ ]:
print(features.shape)
print(returns.shape)
print(features.columns)

(5058, 47674)
(3775, 2167)
MultiIndex([(                      'macd',    1),
            (                      'macd',    2),
            (                      'macd',    3),
            (                      'macd',    4),
            (                      'macd',    5),
            (                      'macd',    6),
            (                      'macd',    7),
            (                      'macd',    8),
            (                      'macd',    9),
            (                      'macd',   10),
            ...
            ('chande_momentum_oscillator', 2158),
            ('chande_momentum_oscillator', 2159),
            ('chande_momentum_oscillator', 2160),
            ('chande_momentum_oscillator', 2161),
            ('chande_momentum_oscillator', 2162),
            ('chande_momentum_oscillator', 2163),
            ('chande_momentum_oscillator', 2164),
            ('chande_momentum_oscillator', 2165),
            ('chande_momentum_oscillator', 2166),
       